# CustomExampleSelector

## 1. Set API Keys

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 2. CustomExampleSelector

현재까지는 유사도 계산시 instruction과 input을 사용하고 있다고 가정해보자.
- 이떄, insturction만 사용해 검색하면 제대로된 유사도 결과가 나오지 않을 수 있음 
- 즉, instruction만 제대로 지정하고 input을 잘못 지정한 경우 예제 선택기에서 올바른 검색이 되지 않을 수 있음

> 커스텀 유사도 계산을 위한 클래스 정의가 필요
>> `from langchain_core.example_selectors.base import BaseExampleSelector` 상속

In [ ]:
from langchain_core.example_selectors.base import BaseExampleSelector
import numpy as np

class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples, embedding_model, search_key="instruction"):
        """
        Example Selector 객체를 초기화 합니다.

        Args:
            examples (list): 예제 데이터 목록
            embedding_model (object): 임베딩 벡터를 게산할 모델 객체
            search_key (str) : 예제에서 입력 값과 비교할 대상의 키
        """
        self.examples = examples
        self.embedding_model = embedding_model
        self.search_key = search_key

        # Calculate all embedding-vectors for examples
        self.example_embeddings = [
            (example, self.embedding_model.embed_query(example[self.search_key]))
            for example in self.examples
        ]
    
    def calc_cosine_similarity(self, vector_1, vector_2):
        """Calculate Cosine Similarity for two embedding-vectors"""
        return np.dot(vector_1, vector_2) / (np.linalg.norm(vector_1) * np.linalg.norm(vector_2))
    
    def add_example(self, example):
        self.examples.append(example)
        self.